# Generic Querying 

Generic querying is an experimental part of `pygen`. This is a method that support a broad set of querying capababilites to an arbitrary Data Model. 
The generic querying takes care of pagination, implementation details in querying, unpacking the results and so on. Will will demonstrate this capabilitiy wiht the `CogniteCore` model

In [1]:
from cognite.client import ClientConfig, CogniteClient

client = CogniteClient(ClientConfig("myName", "myProject", None))

In [2]:
import os

from cognite.client import CogniteClient
from dotenv import load_dotenv

load_dotenv("../../albert.env")

client = CogniteClient.default_oauth_client_credentials(
    project=os.environ["CDF_PROJECT"],
    cdf_cluster=os.environ["CDF_CLUSTER"],
    tenant_id=os.environ["IDP_TENANT_ID"],
    client_id=os.environ["IDP_CLIENT_ID"],
    client_secret=os.environ["IDP_CLIENT_SECRET"],
)

## Setting up method

In [3]:
from pprint import pprint

from cognite.client import data_modeling as dm

from cognite.pygen import _QueryExecutor

In [4]:
executor = _QueryExecutor(client)

## Listing Properties

In [5]:
equipment = dm.ViewId("cdf_cdm", "CogniteEquipment", "v1")

In [6]:
result = executor.list(
    equipment, ["name", "externalId"], sort=dm.InstanceSort(equipment.as_property_ref("name")), limit=3
)

In [7]:
pprint(result)

[{'externalId': 'WMT:23-AE-92527-S1', 'name': '23-AE-92527-S1'},
 {'externalId': 'WMT:23-AE-92527-S2', 'name': '23-AE-92527-S2'},
 {'externalId': 'WMT:23-AE-92527-S3', 'name': '23-AE-92527-S3'}]


## Listing Properties with (reverse) Direct Relation

In [8]:
result = executor.list(
    equipment, ["externalId", {"activities": ["name", "startTime", "endTime", "externalId"]}], limit=1
)

In [9]:
pprint(result)

[{'activities': [{'externalId': '5267267184983944',
                  'name': 'El. *** motor ***/***.'},
                 {'externalId': '7542532148080032',
                  'name': 'DETALJERT INSPEKSJON AV EX UTSTYR'},
                 {'externalId': '3890816994380859',
                  'name': 'El. *** motor ***/***.'},
                 {'externalId': '1994654812359021',
                  'name': 'DETALJERT INSPEKSJON AV EX UTSTYR'},
                 {'externalId': '8975897325791316',
                  'name': '***-årlig visuell inspeksjon'},
                 {'externalId': '5396038908663652', 'name': ''},
                 {'externalId': '5591901218775805',
                  'name': 'Årlig sampling assessment'},
                 {'externalId': '6525225262427728',
                  'name': '36MND Elektro DETALJERT INSPEKSJON AV EX UTSTYR'},
                 {'externalId': '1471919969115313',
                  'name': 'DETALJERT INSPEKSJON AV EX UTSTYR'},
                 {'externalI

In [11]:
result = executor.list(equipment, ["externalId", {"asset": ["externalId", "name"]}], limit=5)

In [12]:
pprint(result)

[{'externalId': 'WMT:23-VA-9110-M01'},
 {'externalId': 'WMT:23-FC-92542'},
 {'externalId': 'WMT:60-EN-9010B+27G2'},
 {'asset': [{'externalId': 'WMT:23-1ST STAGE COMP DRY GAS SEAL SYS-PH',
             'name': '23-1ST STAGE COMP DRY GAS SEAL SYS-PH'}],
  'externalId': 'WMT:23-PSV-96173'},
 {'externalId': 'WMT:23-FO-96195'}]


## Listing wiht Edges

Note that there are no edges connected to any of the core model views. However, if the `asset` property of equipment was an edge 
the above query would look like

In [ ]:
result = executor.list(equipment, ["externalId", {"asset": ["type", {"node": ["externalId", "name"]}]}], limit=5)

The above query would return the `externalID` and `name` of each asset, in addition to the `type` of the edge connecting the equipment and the asset.

## Aggregations

### Numeric Aggregation

In [15]:
result = executor.aggregate(equipment, aggregates=dm.aggregations.Count("externalId"))

In [16]:
pprint(result)

{'count': {'externalId': 1094}}


### Group by

In [17]:
result = executor.aggregate(equipment, aggregates=dm.aggregations.Count("externalId"), group_by="manufacturer")

In [18]:
pprint(result)

[{'count': {'externalId': 1041}, 'group': {'manufacturer': '1686'}},
 {'count': {'externalId': 31}, 'group': {'manufacturer': ''}},
 {'count': {'externalId': 11}, 'group': {'manufacturer': '1350'}},
 {'count': {'externalId': 9}, 'group': {'manufacturer': '1327'}},
 {'count': {'externalId': 1}, 'group': {'manufacturer': '1041'}},
 {'count': {'externalId': 1}, 'group': {'manufacturer': '1314'}}]


### Histogram

There are no number properties in the `CogniteCore` model with the excpetion of Annotations and 3D. If `CogniteActivity` has the integere `activityCount`, 
the histogram aggregatoin would look like this

In [20]:
activity = dm.ViewId("cdf_cdm", "CogniteActivity", "v1")

In [ ]:
result = executor.aggregate(activity, aggregates=dm.aggregations.Histogram("activityCount", interval=10))

## Search

In [22]:
result = executor.search(equipment, query="23", limit=5)

In [23]:
pprint(result)

[{'createdTime': 1730210932388,
  'description': 'ACCUMULATOR FOR 23-FV-92543',
  'externalId': 'WMT:23-FVX-92543',
  'files': [],
  'instanceType': 'node',
  'lastUpdatedTime': 1730210932388,
  'manufacturer': '',
  'name': '23-FVX-92543',
  'serialNumber': '1000000000042272',
  'sourceId': '743717',
  'sourceUpdatedUser': '8137',
  'space': 'springfield_instances',
  'version': 1},
 {'createdTime': 1730210934298,
  'description': 'DRAIN POT FOR 23-PDT-96167',
  'externalId': 'WMT:23-TX-96167',
  'files': [],
  'instanceType': 'node',
  'lastUpdatedTime': 1730210934298,
  'manufacturer': '',
  'name': '23-TX-96167',
  'serialNumber': '1000000000033347',
  'sourceId': '741832',
  'sourceUpdatedUser': '8137',
  'space': 'springfield_instances',
  'version': 1},
 {'createdTime': 1730210931078,
  'description': 'VRD - 1ST STAGE COMPRESSOR OIL SEPARATOR W/ FAN',
  'externalId': 'WMT:23-VA-9110-M01',
  'files': [],
  'instanceType': 'node',
  'lastUpdatedTime': 1730210931078,
  'manufacture

## Search with nested properties

In [31]:
result = executor.search(
    equipment,
    properties=["externalId", "manufacturer", {"activities": ["name", "externalId"]}],
    query="23",
    search_properties=["name"],
    limit=2,
)
result

[{'externalId': 'WMT:23-VA-9110-M01',
  'manufacturer': '1686',
  'activities': [{'externalId': '5267267184983944',
    'name': 'El. *** motor ***/***.'},
   {'externalId': '7542532148080032',
    'name': 'DETALJERT INSPEKSJON AV EX UTSTYR'},
   {'externalId': '3890816994380859', 'name': 'El. *** motor ***/***.'},
   {'externalId': '1994654812359021',
    'name': 'DETALJERT INSPEKSJON AV EX UTSTYR'},
   {'externalId': '8975897325791316', 'name': '***-årlig visuell inspeksjon'},
   {'externalId': '5396038908663652', 'name': ''},
   {'externalId': '5591901218775805', 'name': 'Årlig sampling assessment'},
   {'externalId': '6525225262427728',
    'name': '36MND Elektro DETALJERT INSPEKSJON AV EX UTSTYR'},
   {'externalId': '1471919969115313',
    'name': 'DETALJERT INSPEKSJON AV EX UTSTYR'},
   {'externalId': '6341564007115449',
    'name': '*** MND ELEKTRO. Ettersyn av ***'},
   {'externalId': '3752022002179906',
    'name': '36MND Elektro DETALJERT INSPEKSJON AV EX UTSTYR'},
   {'extern